# BTC Wave Detection Testing Notebook

This notebook contains the main function from the updated wave detection system for debugging and parameter tuning.

## Features:
- RSI-driven extrema detection
- 5-wave and 7-wave pattern recognition
- Intelligent overlapping wave merging
- Enhanced visualization

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the project directory to path
sys.path.append(os.getcwd())

# Import from our modules
from core.wave_detection import find_downtrend_wave_patterns, handle_overlapping_waves
from core.rsi_analysis import find_extremas_with_rsi
from utils.technical_indicators import calculate_rsi
from visualization.plotting import plot_individual_wave, plot_overview_chart

In [ ]:
def load_data(file_path):
    """
    加载 CSV 数据文件
    """
    try:
        data = pd.read_csv(file_path)
        print(f"成功读取文件: {file_path}")
    except FileNotFoundError:
        print(f"错误: 文件 '{file_path}' 未找到。")
        return None
        
    if 'datetime' not in data.columns or 'close' not in data.columns:
        print("错误: CSV文件必须包含 'datetime' 和 'close' 两列。")
        return None
        
    data['datetime'] = pd.to_datetime(data['datetime'])
    data.set_index('datetime', inplace=True)
    return data['close']

# Load the data
file_path = 'BTC.csv'  # Adjust path if needed
close_prices = load_data(file_path)

if close_prices is not None:
    print(f"数据加载成功: {len(close_prices)} 条记录")
    print(f"时间范围: {close_prices.index.min().date()} 到 {close_prices.index.max().date()}")
    print(f"价格范围: ${close_prices.min():.2f} - ${close_prices.max():.2f}")
else:
    print("数据加载失败")

In [ ]:
# Configure analysis parameters (you can adjust these for debugging)
rsi_period = 14
rsi_drop_threshold = 10  # Try different values: 8, 10, 12, 15
rsi_rise_ratio = 1/3     # Try different values: 1/4, 1/3, 1/2

print(f"分析参数:")
print(f"  - RSI 周期: {rsi_period}")
print(f"  - RSI 下跌阈值: {rsi_drop_threshold}")
print(f"  - RSI 上涨比例: {rsi_rise_ratio:.3f}")

In [ ]:
# Calculate RSI
print("正在计算 RSI 指标...")
rsi_series = calculate_rsi(close_prices, period=rsi_period)
print(f"RSI 计算完成, 有效数据点: {len(rsi_series.dropna())}")

# Quick RSI statistics
rsi_clean = rsi_series.dropna()
print(f"RSI 统计:")
print(f"  - 平均值: {rsi_clean.mean():.2f}")
print(f"  - 最小值: {rsi_clean.min():.2f}")
print(f"  - 最大值: {rsi_clean.max():.2f}")

In [ ]:
# Find extremas using RSI
print("\n=== RSI 驱动的极值点识别 ===")
extremas, all_trigger_points = find_extremas_with_rsi(
    close_prices, 
    rsi_period=rsi_period, 
    rsi_drop_threshold=rsi_drop_threshold, 
    rsi_rise_ratio=rsi_rise_ratio
)

print(f"\n识别结果:")
print(f"  - 极值点数量: {len(extremas)}")
print(f"  - 触发点数量: {len(all_trigger_points)}")

# Show trigger point breakdown
drop_triggers = [p for p in all_trigger_points if p['type'] == 'drop']
rise_triggers = [p for p in all_trigger_points if p['type'] == 'rise']
print(f"  - Drop 触发点: {len(drop_triggers)}")
print(f"  - Rise 触发点: {len(rise_triggers)}")

In [ ]:
# Find wave patterns
print("\n=== 波浪模式识别 ===")
all_waves = find_downtrend_wave_patterns(close_prices, extremas, length=6)

print(f"初始波浪检测:")
print(f"  - 总波浪数: {len(all_waves)}")

# Breakdown by type
strict_waves = [w for w in all_waves if w['type'] == 'strict']
relaxed_waves = [w for w in all_waves if w['type'] == 'relaxed']
print(f"  - Strict 波浪: {len(strict_waves)}")
print(f"  - Relaxed 波浪: {len(relaxed_waves)}")

In [ ]:
# Handle overlapping waves
print("\n=== 重叠波浪处理 ===")
final_waves, merged_waves = handle_overlapping_waves(all_waves, close_prices, extremas)

print(f"处理结果:")
print(f"  - 原始波浪: {len(all_waves)}")
print(f"  - 最终独立波浪: {len(final_waves)}")
print(f"  - 合并波浪 (7-wave): {len(merged_waves)}")
print(f"  - 总有效波浪: {len(final_waves) + len(merged_waves)}")

In [ ]:
# Create overview chart
print("\n=== 生成概览图表 ===")

# Prepare wave data for plotting
wave_data = {
    'final_waves': final_waves,
    'merged_waves': merged_waves
}

start_date_all = close_prices.index.min().date()
end_date_all = close_prices.index.max().date()

plot_overview_chart(
    close_prices, 
    wave_data,
    start_date_all, 
    end_date_all, 
    rsi_drop_threshold, 
    rsi_rise_ratio
)

In [ ]:
# Generate detailed charts for each wave
print("\n=== 生成详细波浪图表 ===")

all_processed_waves = final_waves + merged_waves

if all_processed_waves:
    print(f"将生成 {len(all_processed_waves)} 个详细图表...")
    
    for i, wave in enumerate(all_processed_waves):
        wave_indices = wave['indices']
        wave_type = wave['type']
        wave_start_date = close_prices.index[wave_indices[0]]
        wave_end_date = close_prices.index[wave_indices[-1]]
        
        # Get relevant trigger points for this wave
        wave_trigger_points = [
            p for p in all_trigger_points 
            if p['date'] >= wave_start_date and p['date'] <= wave_end_date
        ]
        
        print(f"\n绘制波浪 {i+1}/{len(all_processed_waves)}: {wave_type} ({len(wave_indices)} 点)")
        print(f"  时间范围: {wave_start_date.date()} - {wave_end_date.date()}")
        print(f"  相关触发点: {len(wave_trigger_points)}")
        
        plot_individual_wave(
            close_prices, 
            rsi_series, 
            wave_indices, 
            wave_trigger_points, 
            plot_range_days=15, 
            wave_number=i+1, 
            wave_type=wave_type
        )
else:
    print("没有检测到有效波浪。")

In [ ]:
# Analysis summary
print("\n" + "="*50)
print("          波浪检测分析总结")
print("="*50)

print(f"\n数据概况:")
print(f"  - 数据文件: {file_path}")
print(f"  - 数据点数: {len(close_prices)}")
print(f"  - 时间跨度: {(close_prices.index.max() - close_prices.index.min()).days} 天")

print(f"\n分析参数:")
print(f"  - RSI 周期: {rsi_period}")
print(f"  - Drop 阈值: {rsi_drop_threshold}")
print(f"  - Rise 比例: {rsi_rise_ratio:.3f}")

print(f"\n检测结果:")
print(f"  - 极值点: {len(extremas)}")
print(f"  - 触发点: {len(all_trigger_points)}")
print(f"  - 原始波浪: {len(all_waves)}")
print(f"  - 最终独立波浪: {len(final_waves)}")
print(f"  - 合并波浪: {len(merged_waves)}")
print(f"  - 总有效波浪: {len(final_waves) + len(merged_waves)}")

if all_processed_waves:
    print(f"\n波浪详情:")
    for i, wave in enumerate(all_processed_waves):
        wave_indices = wave['indices']
        wave_type = wave['type']
        start_date = close_prices.index[wave_indices[0]].date()
        end_date = close_prices.index[wave_indices[-1]].date()
        duration = (close_prices.index[wave_indices[-1]] - close_prices.index[wave_indices[0]]).days
        
        print(f"  {i+1}. {wave_type.upper()} ({len(wave_indices)}点) - {start_date} to {end_date} ({duration}天)")

print("\n分析完成!")

In [ ]:
# Debugging and parameter testing section
print("\n" + "="*40)
print("         调试和参数测试区域")
print("="*40)
print("\n在这里你可以:")
print("1. 修改上面的参数重新运行分析")
print("2. 测试不同的 rsi_drop_threshold 值")
print("3. 测试不同的 rsi_rise_ratio 值")
print("4. 检查特定日期范围的数据")
print("5. 分析触发点和极值点的分布")

# Quick parameter testing function
def quick_test_parameters(drop_threshold, rise_ratio):
    """
    快速测试不同参数组合
    """
    print(f"\n测试参数: drop_threshold={drop_threshold}, rise_ratio={rise_ratio:.3f}")
    
    # Run analysis with new parameters
    test_extremas, test_triggers = find_extremas_with_rsi(
        close_prices, 
        rsi_period=14, 
        rsi_drop_threshold=drop_threshold, 
        rsi_rise_ratio=rise_ratio
    )
    
    test_waves = find_downtrend_wave_patterns(close_prices, test_extremas, length=6)
    test_final, test_merged = handle_overlapping_waves(test_waves, close_prices, test_extremas)
    
    print(f"结果: {len(test_extremas)} 极值点, {len(test_waves)} 原始波浪, {len(test_final)+len(test_merged)} 最终波浪")
    
    return len(test_extremas), len(test_waves), len(test_final)+len(test_merged)

# Example parameter testing (uncomment to run)
# print("\n参数敏感性测试:")
# for threshold in [8, 10, 12, 15]:
#     for ratio in [0.2, 0.25, 0.33, 0.5]:
#         quick_test_parameters(threshold, ratio)